# Microestructuras y sistemas de Trading
## Laboratorio 1

## Equipo C
### Participantes:
>Paulina Milenka Glez Manrique, Andres García Ruvalcaba, Mariana López Correa,Juan López Pelayo.

In [1]:
import quandl 
import pandas as pd

In [2]:
quandl.ApiConfig.api_key = "6ARMqMT_1jbzJ71XyRCx"

In [16]:
excel_etf = 'IVW.xlsx'
data = pd.read_excel(excel_etf, sheet_name=0, header=0, sep=',', index_col=None, parse_dates=False, skip_blank_lines=True)
data.head()

,Ticker,Nombre,Clase de activo,Peso (%),Precio,Acciones,Valor de mercado,Valor nominal,Sector,SEDOL,ISIN,Intercambio
0,MSFT,MICROSOFT CORP,Equity,8.00530,135.74,13603103,1.846485e+09,1.846485e+09,Tecnología de la Información,2588173,US5949181045,NASDAQ
1,AMZN,AMAZON COM INC,Equity,5.60768,1761.83,734155,1.293456e+09,1.293456e+09,Consumo discrecional,2000019,US0231351067,NASDAQ
2,FB,FACEBOOK CLASS A INC,Equity,3.35236,181.30,4265020,7.732481e+08,7.732481e+08,Comunicación,B7TL820,US30303M1027,NASDAQ
3,GOOG,ALPHABET INC CLASS C,Equity,2.75459,1167.84,544054,6.353680e+08,6.353680e+08,Comunicación,BYY88Y7,US02079K1079,NASDAQ
4,GOOGL,ALPHABET INC CLASS A,Equity,2.69820,1170.82,531560,6.223611e+08,6.223611e+08,Comunicación,BYVY8G0,US02079K3059,NASDAQ
